In [2]:
import os
from pathlib import Path

In [3]:
data_dir = "/beegfs/CMIP6/arctic-cmip6/cmip6_fwi"

In [1]:
# model subdirectories contain one NC file per year, named like cffdrs_<model>_YYYY.nc
# for each model subdirectory, list the NC files and check years
# for each model subdirectory, print start year, end year, number of years, and true/false if there are any missing years in the sequence


def check_model_years(model_dir):
    model_path = Path(model_dir)
    if not model_path.is_dir():
        print(f"{model_dir} is not a valid directory.")
        return

    nc_files = sorted(model_path.glob("cffdrs_*.nc"))
    if not nc_files:
        print(f"No NC files found in {model_dir}.")
        return

    years = []
    for nc_file in nc_files:
        filename = nc_file.name
        try:
            year = int(filename.split("_")[-1].split(".")[0])
            years.append(year)
        except ValueError:
            print(f"Filename {filename} does not conform to expected format.")

    if not years:
        print(f"No valid years found in filenames in {model_dir}.")
        return

    start_year = min(years)
    end_year = max(years)
    num_years = len(years)
    expected_years = set(range(start_year, end_year + 1))
    actual_years = set(years)
    missing_years = expected_years - actual_years
    has_missing_years = len(missing_years) > 0

    print(f"Model Directory: {model_dir}")
    print(f"Start Year: {start_year}")
    print(f"End Year: {end_year}")
    print(f"Number of Years: {num_years}")
    print(f"Missing Years: {sorted(missing_years)}")
    print(f"Has Missing Years: {has_missing_years}")
    print()

In [4]:
# run the check for each model subdirectory in the data directory

for model_subdir in os.listdir(data_dir):
    model_path = os.path.join(data_dir, model_subdir)
    check_model_years(model_path)

FileNotFoundError: [Errno 2] No such file or directory: '/beegfs/CMIP6/arctic-cmip6/cmip6_fwi'